<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd 
import numpy as np

In [2]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/基金產品列表.csv', encoding='CP950')

In [3]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [4]:
fund_df = fund_df[fund_df['RISKLEVEL']!=0][['CNAME','FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME']]
fund_df['SECTOR_NAME'] = fund_df['SECTOR_NAME'].fillna('NA')
fund_df = fund_df[~fund_df['ASSETCAT_CODE'].isna()]

In [5]:
corpus = fund_df.astype(str).values.tolist()

In [9]:
corpus[0:5]

[['施羅德２０２２到期新興市場主權債券基金（美元）－累積型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '03_不配',
  'BOND',
  'NA'],
 ['施羅德２０２２到期新興市場主權債券基金（美元）－配息型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  'NA'],
 ['新光創新科技基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '科技'],
 ['新光店頭基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市'],
 ['新光台灣富貴基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市']]

In [10]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [11]:
model = Word2Vec(corpus, size=50, iter=50,window=5, sg=1,negative=20,min_count=1)


In [13]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [24]:
np.savetxt("fund_embedd.tsv", model.wv[fund_df['CNAME'].tolist()], delimiter="\t")


In [25]:
fund_df.to_csv('fund_embedd_metadata.tsv', sep = '\t',index=False)

In [26]:
fund_df.shape

(2650, 6)

In [27]:
from google.colab import files
files.download('/content/fund_embedd_metadata.tsv') 
files.download('/content/fund_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
! wget https://www.rahimikia.com/FinText/FinText_Word2Vec_Skip-gram.zip

--2021-10-26 13:36:09--  https://www.rahimikia.com/FinText/FinText_Word2Vec_Skip-gram.zip
Resolving www.rahimikia.com (www.rahimikia.com)... 69.163.228.90
Connecting to www.rahimikia.com (www.rahimikia.com)|69.163.228.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6291927333 (5.9G) [application/zip]
Saving to: ‘FinText_Word2Vec_Skip-gram.zip’

FinText_Word2Vec_Sk 100%[===================>]   5.86G  24.2MB/s    in 4m 3s   

2021-10-26 13:40:12 (24.7 MB/s) - ‘FinText_Word2Vec_Skip-gram.zip’ saved [6291927333/6291927333]



In [29]:
!unzip /content/FinText_Word2Vec_Skip-gram.zip

Archive:  /content/FinText_Word2Vec_Skip-gram.zip
   creating: FinText_Word2Vec_Skip-gram/
  inflating: FinText_Word2Vec_Skip-gram/Word_Embedding_2000_2015  
  inflating: FinText_Word2Vec_Skip-gram/README.txt  
  inflating: FinText_Word2Vec_Skip-gram/Word_Embedding_2000_2015.pkl  
  inflating: FinText_Word2Vec_Skip-gram/Word_Embedding_2000_2015.wv.vectors.npy  
  inflating: FinText_Word2Vec_Skip-gram/Word_Embedding_2000_2015.trainables.syn1neg.npy  


In [1]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api 

In [ ]:
Google_WE = api.load('word2vec-google-news-300') # Google (skip-gram)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [2]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_Word2Vec_Skip-gram/Word_Embedding_2000_2015')

In [38]:

prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [39]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]

In [40]:
product_embedding.shape

(18, 300)

In [41]:
np.savetxt("product_embedd.tsv", product_embedding, delimiter="\t")


In [42]:
pd.DataFrame({'product':prod}).to_csv('prod_embedd_metadata.tsv', sep = '\t',index=False,header=False)

In [43]:
from google.colab import files
files.download('/content/prod_embedd_metadata.tsv') 
files.download('/content/product_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>